In [1]:
import tmscoring

de_novo_filepath = '../../datasets/alphafold_test_output/antibody_de_novo.pdb'
original_filepath = '../../datasets/alphafold_test_output/antibody_real.pdb'

alignment = tmscoring.TMscoring(de_novo_filepath, original_filepath)
alignment.optimise()
print(alignment.tmscore(**alignment.get_current_values()))
print(alignment.tmscore_samples(**alignment.get_current_values()))
print(alignment.rmsd(**alignment.get_current_values()))

0.9961657282211568
[0.9065317  0.97646536 0.99066769 0.99943629 0.99890903 0.99974941
 0.99919408 0.9990635  0.99304587 0.99879329 0.99946023 0.99920663
 0.99860702 0.99953093 0.99966989 0.9994749  0.99404764 0.99794738
 0.99951838 0.99976513 0.9989943  0.99973341 0.99947068 0.99996409
 0.99173974 0.97946622 0.97217669 0.99324986 0.99753606 0.99450997
 0.995163   0.99690499 0.99958994 0.99958745 0.9981017  0.99974747
 0.99965754 0.99974237 0.9999491  0.99945208 0.98905179 0.94714656
 0.98124665 0.99985969 0.99957074 0.99821128 0.998696   0.99876352
 0.99969081 0.99919732 0.99982679 0.99936568 0.99979322 0.99961682
 0.99966988 0.99954144 0.999707   0.99956972 0.99842408 0.99288001
 0.99014968 0.99701254 0.99750572 0.9967157  0.99644031 0.99818758
 0.99972847 0.99978298 0.99955749 0.99934215 0.99991833 0.99986573
 0.99974039 0.9997296  0.99972779 0.99965105 0.99949743 0.99973922
 0.99961838 0.99994651 0.99996183 0.99960863 0.99936582 0.99960562
 0.99987396 0.99950064 0.99793754 0.9968109

Separability

In [2]:
import requests

class annotate():
    """
    class `annotate`. 
    
    Initiator `__init__` has 2 parameters:
    
    :param aaseq: STRING: A single-letter, amino acid sequence corresponding to the complete VH or VL chain. Both uppercase and lowercase are accepted. 
    
    :param scheme: STRING: "kabat", "chothia", "contact", or "imgt". Must be in lowercase
    
    Class has 3 methods. `retrieve()`: retrieves numbered seqs from Abnum website, then sends it to method `analyze` to determine the FR and CDR regions, and to `output() ` to print the result and return a list of 2 dictionaries, the first of which contains to region:seq pairs, the second of which contains number:residue pairs. 
    
    """
    
    def __init__(self, aaseq, scheme):
        
        self.aaseq=aaseq
        self.scheme=scheme
    
    def __repr__(self):
        return "Annotation of VH or VL sequence using Kabat, Chothia, Contact, or IMGT scheme"
    
    def output(self, chain, lst, regionlst):
        """
        Prints the FR and CDR regions and their corresponding seq. It returns a `list` of 2 `dict`. 
        
        :param chain: STRING, either "H" or "L" in uppercase
        :param lst:  LIST, a list of residue and their corresponding numbers in kabat or chothia scheme
        :param regionlst: LIST, a list of peptides, each corresponds to a FR or CDR region
        :return: LIST, a list of 2 `dict`, The first dict consists of region: seq pairs. The second dict consists of number:residue pairs.
        
        """
        
        self.chain=chain
        self.lst=lst
        self.regionlst=regionlst

        self.regiondict, self.numberdict={}, {}
        
        for i in range (0, len(self.lst), 2):
            self.numberdict[self.lst[i]]=self.lst[i+1]
        
        
        if self.scheme=="kabat":
            print("Annotation in Kabat scheme:")
        elif self.scheme=="chothia":
            print("Annotation in Chothia scheme:")
        elif self.scheme=="contact":
            print("Annotation in Contact scheme:")
        else:
            print("Annotation in IMGT scheme:")
        
        if self.chain=="L":
            print("L-FR1:  ", self.regionlst[0])
            print("L-CDR1: ", self.regionlst[1])
            print("L-FR2:  ", self.regionlst[2])
            print("L-CDR2: ", self.regionlst[3])
            print("L-FR3:  ", self.regionlst[4])
            print("L-CDR3: ", self.regionlst[5])
            print("L-FR4:  ", self.regionlst[6])
            
            for region, seq in zip(["L-FR1", "L-CDR1", "L-FR2","L-CDR2", "L-FR3", "L-CDR3", "L-FR4"], self.regionlst):
                self.regiondict[region]=seq
            
            return [self.regiondict, self.numberdict]
                
        else:
            print("H-FR1:  ", self.regionlst[0])
            print("H-CDR1: ", self.regionlst[1])
            print("H-FR2:  ", self.regionlst[2])
            print("H-CDR2: ", self.regionlst[3])
            print("H-FR3:  ", self.regionlst[4])
            print("H-CDR3: ", self.regionlst[5])
            print("H-FR4:  ", self.regionlst[6])
            
            for region, seq in zip(["H-FR1", "H-CDR1", "H-FR2","H-CDR2", "H-FR3", "H-CDR3", "H-FR4"], self.regionlst):
                self.regiondict[region]=seq
            
            return [self.regiondict, self.numberdict]
            
        
        
        

    
    def analyze(self,chain, lst):
        """
        Define CDR and FR regions based on the numbered sequence returned from website
        
        :param chain: STRING, "H" or "L" in uppercase
        :param lst: LIST, a list of residue and their corresponding numbers in kabat or chothia scheme
        :return: LIST, a list of strings, where each string is a peptide corresponding to the a region, in the order of: FR1, CDR1, FR2, CDR2, FR3, CDR3, FR4
        
        :raises: `ValueError` if any of the FR or CDR region is missing
        
        """
        
        self.chain=chain
        self.lst=lst
        if self.chain=="L":
            self.L_FR1, self.L_CDR1, self.L_FR2, self.L_CDR2, self.L_FR3, self.L_CDR3, self.L_FR4=["" for i in range (0, 7)]
            
            try:
                if self.scheme in ["kabat", "chothia"]:
                    self.L_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("L24"), 2)])
                    self.L_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("L24"), self.lst.index("L35"), 2)])
                    self.L_FR2="".join([self.lst[i+1] for i in range (self.lst.index("L35"), self.lst.index("L50"), 2)])
                    self.L_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("L50"), self.lst.index("L57"), 2)])
                    self.L_FR3="".join([self.lst[i+1] for i in range (self.lst.index("L57"), self.lst.index("L89"), 2)])
                    self.L_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("L89"), self.lst.index("L98"), 2)])
                    self.L_FR4="".join([self.lst[i+1] for i in range (self.lst.index("L98"), len(self.lst), 2)])
                                    
                elif self.scheme =="contact": 
                    self.L_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("L30"), 2)])
                    self.L_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("L30"), self.lst.index("L37"), 2)])
                    self.L_FR2="".join([self.lst[i+1] for i in range (self.lst.index("L37"), self.lst.index("L46"), 2)])
                    self.L_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("L46"), self.lst.index("L56"), 2)])
                    self.L_FR3="".join([self.lst[i+1] for i in range (self.lst.index("L56"), self.lst.index("L89"), 2)])
                    self.L_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("L89"), self.lst.index("L97"), 2)])
                    self.L_FR4="".join([self.lst[i+1] for i in range (self.lst.index("L97"), len(self.lst), 2)])
                                    
                else: #IMGT scheme
                    self.L_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("L27"), 2)])
                    self.L_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("L27"), self.lst.index("L33"), 2)])
                    self.L_FR2="".join([self.lst[i+1] for i in range (self.lst.index("L33"), self.lst.index("L50"), 2)])
                    self.L_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("L50"), self.lst.index("L52"), 2)])
                    self.L_FR3="".join([self.lst[i+1] for i in range (self.lst.index("L52"), self.lst.index("L89"), 2)])
                    self.L_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("L89"), self.lst.index("L98"), 2)])
                    self.L_FR4="".join([self.lst[i+1] for i in range (self.lst.index("L98"), len(self.lst), 2)])
                
                return [self.L_FR1, self.L_CDR1, self.L_FR2, self.L_CDR2, self.L_FR3, self.L_CDR3, self.L_FR4] 

            except ValueError:
                print("Unable to retrieve complete V region. Make sure the sequence has complete V region")
            except:
                print("An error occured")
        else:
            self.H_FR1, self.H_CDR1, self.H_FR2, self.H_CDR2, self.H_FR3, self.H_CDR3, self.H_FR4=["" for i in range (0, 7)]
            try:
                if self.scheme=="kabat":
                    self.H_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("H31"), 2)])
                    self.H_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("H31"), self.lst.index("H36"), 2)])
                    self.H_FR2="".join([self.lst[i+1] for i in range (self.lst.index("H36"), self.lst.index("H50"), 2)])
                    self.H_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("H50"), self.lst.index("H66"), 2)])
                    self.H_FR3="".join([self.lst[i+1] for i in range (self.lst.index("H66"), self.lst.index("H95"), 2)])
                    self.H_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("H95"), self.lst.index("H103"), 2)])
                    self.H_FR4="".join([self.lst[i+1] for i in range (self.lst.index("H103"), len(self.lst), 2)])            
            
                elif self.scheme=="chothia":
                    self.H_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("H26"), 2)])
                    self.H_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("H26"), self.lst.index("H33"), 2)])
                    self.H_FR2="".join([self.lst[i+1] for i in range (self.lst.index("H33"), self.lst.index("H52"), 2)])
                    self.H_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("H52"), self.lst.index("H57"), 2)])
                    self.H_FR3="".join([self.lst[i+1] for i in range (self.lst.index("H57"), self.lst.index("H95"), 2)])
                    self.H_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("H95"), self.lst.index("H103"), 2)])
                    self.H_FR4="".join([self.lst[i+1] for i in range (self.lst.index("H103"), len(self.lst), 2)])

                elif self.scheme=="contact":
                    self.H_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("H30"), 2)])
                    self.H_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("H30"), self.lst.index("H36"), 2)])
                    self.H_FR2="".join([self.lst[i+1] for i in range (self.lst.index("H36"), self.lst.index("H47"), 2)])
                    self.H_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("H47"), self.lst.index("H59"), 2)])
                    self.H_FR3="".join([self.lst[i+1] for i in range (self.lst.index("H59"), self.lst.index("H93"), 2)])
                    self.H_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("H93"), self.lst.index("H102"), 2)])
                    self.H_FR4="".join([self.lst[i+1] for i in range (self.lst.index("H102"), len(self.lst), 2)])
                                        
                else: #IMGT scheme
                    self.H_FR1="".join([self.lst[i+1] for i in range (0, self.lst.index("H26"), 2)])
                    self.H_CDR1="".join([self.lst[i+1] for i in range (self.lst.index("H26"), self.lst.index("H34"), 2)])
                    self.H_FR2="".join([self.lst[i+1] for i in range (self.lst.index("H34"), self.lst.index("H51"), 2)])
                    self.H_CDR2="".join([self.lst[i+1] for i in range (self.lst.index("H51"), self.lst.index("H58"), 2)]) #51>57 (instead of 56)
                    self.H_FR3="".join([self.lst[i+1] for i in range (self.lst.index("H58"), self.lst.index("H93"), 2)])
                    self.H_CDR3="".join([self.lst[i+1] for i in range (self.lst.index("H93"), self.lst.index("H103"), 2)])
                    self.H_FR4="".join([self.lst[i+1] for i in range (self.lst.index("H103"), len(self.lst), 2)])                    
                
                return [self.H_FR1, self.H_CDR1, self.H_FR2, self.H_CDR2, self.H_FR3, self.H_CDR3, self.H_FR4]                    

            except ValueError:
                print("Unable to retrieve complete V region. Make sure the sequence has complete V region")
            except:
                print("An error occured in the `analyze()` method")
        
    def retrieve (self):
        """
        Retrieve numbered residues from Abnum website
        
        :return: returns same object from the `output()` method. 
        
        :raises: `ValueError` if input scheme is not among "kabat", "chothia", "contact", and "imgt"
        
        """
        
        self.url="http://www.bioinf.org.uk/abs/abnum/abnum.cgi"
        
        try: 
            if self.scheme not in ["kabat", "chothia", "contact", "imgt", "martin"]:
                raise Exception
            
        except ValueError:
            print("Incorrect scheme mode. Must be one of the following (lowercase): kabat, chothia, contact, imgt")
        
        else:
            if self.scheme=="kabat":
                self.sche="-k"
            else:
                self.sche="-c"
        
        try:
            self.d={"plain":1, "scheme":self.sche, "aaseq":self.aaseq}
            self.myPage=requests.get(self.url, params=self.d)
            self.text=self.myPage.text
            self.lst=self.text.split()
                
            if len(self.lst)>1:
                self.chain=self.lst[0][0]
                self.result=self.output(self.chain, self.lst, self.analyze(self.chain, self.lst))
                return self.result
            else:
                print("No annotation retrieved. Did you enter the complete VH or VL sequence?")
        except:
            print("An error occured in the `retrieve()` method")

In [8]:
from Bio import SeqIO, SeqRecord
import os, contextlib
import pandas as pd
from tqdm import tqdm
import warnings
import json
from Bio import Align

small_ft_ground_truths_path = "../../datasets/outputs/small_ft_full_beam/test/processed"

aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.open_gap_score = -0
aligner.extend_gap_score = -0
scheme="martin"

cdr1s = []
cdr2s = []
cdr3s = []

heavy_chains = 0
light_chains = 0

with open(small_ft_ground_truths_path) as handle:
    for k, record in enumerate(tqdm(handle.readlines())):
        if k % 3 == 0:
            print(record)
            seq=annotate(record,scheme)
            with open(os.devnull, 'w') as devnull:
                with contextlib.redirect_stdout(devnull):
                    # f()
                    result=seq.retrieve()
            print(result)
            if result[0].get("L-CDR1") is not None:
                cdr1s.append(result[0]["L-CDR1"])
                cdr2s.append(result[0]["L-CDR2"])
                cdr3s.append(result[0]["L-CDR3"])
                light_chains += 1
            else:
                cdr1s.append(result[0]["H-CDR1"])
                cdr2s.append(result[0]["H-CDR2"])
                cdr3s.append(result[0]["H-CDR3"])
                heavy_chains += 1

print(f"Light chains: {light_chains}")
print(f"Heavy chains: {heavy_chains}")
print(f"Ratio heavy to light: {heavy_chains/light_chains}")
json.dump(cdr1s, open("cdr1s.json", "w"))
json.dump(cdr2s, open("cdr2s.json", "w"))
json.dump(cdr3s, open("cdr3s.json", "w"))

  0%|          | 0/3000 [00:00<?, ?it/s]

IVMTQTPLSLSVSPGETASISCKANSNGNTYLVWYRQKPGQSPQRLINLVSNRDPGVPDRFSGSGSGTDFTLRISRVEADDAGVYYCGQGIQVPVTFGQGTKLEIK



  0%|          | 1/3000 [00:00<13:15,  3.77it/s]

[{'L-FR1': 'IVMTQTPLSLSVSPGETASISCKAN', 'L-CDR1': 'SNGNTY', 'L-FR2': 'LVWYRQKPGQSPQRLIN', 'L-CDR2': 'LV', 'L-FR3': 'SNRDPGVPDRFSGSGSGTDFTLRISRVEADDAGVYYC', 'L-CDR3': 'GQGIQVPVT', 'L-FR4': 'FGQGTKLEIK'}, {'L2': 'I', 'L3': 'V', 'L4': 'M', 'L5': 'T', 'L6': 'Q', 'L7': 'T', 'L8': 'P', 'L9': 'L', 'L10': 'S', 'L11': 'L', 'L12': 'S', 'L13': 'V', 'L14': 'S', 'L15': 'P', 'L16': 'G', 'L17': 'E', 'L18': 'T', 'L19': 'A', 'L20': 'S', 'L21': 'I', 'L22': 'S', 'L23': 'C', 'L24': 'K', 'L25': 'A', 'L26': 'N', 'L27': 'S', 'L28': 'N', 'L29': 'G', 'L30': 'N', 'L31': 'T', 'L32': 'Y', 'L33': 'L', 'L34': 'V', 'L35': 'W', 'L36': 'Y', 'L37': 'R', 'L38': 'Q', 'L39': 'K', 'L40': 'P', 'L41': 'G', 'L42': 'Q', 'L43': 'S', 'L44': 'P', 'L45': 'Q', 'L46': 'R', 'L47': 'L', 'L48': 'I', 'L49': 'N', 'L50': 'L', 'L51': 'V', 'L52': 'S', 'L53': 'N', 'L54': 'R', 'L55': 'D', 'L56': 'P', 'L57': 'G', 'L58': 'V', 'L59': 'P', 'L60': 'D', 'L61': 'R', 'L62': 'F', 'L63': 'S', 'L64': 'G', 'L65': 'S', 'L66': 'G', 'L67': 'S', 'L68': 'G', 

  0%|          | 4/3000 [00:00<05:47,  8.63it/s]

...

[{'H-FR1': 'EGQLAESGGDLVNPGGSLRLSCVAS', 'H-CDR1': 'GFSFSHFD', 'H-FR2': 'MNWVRQAPGKGLQWVAT', 'H-CDR2': 'IPADGSDT', 'H-FR3': 'YYTGAVRGRFTLSRDNAKNTLYLQMNSLRAEDTAVYYC', 'H-CDR3': 'ARRIVAQSPNFDY', 'H-FR4': 'WGQGTLVTVSS'}, {'H1': 'E', 'H2': 'G', 'H3': 'Q', 'H4': 'L', 'H5': 'A', 'H6': 'E', 'H7': 'S', 'H8': 'G', 'H9': 'G', 'H10': 'D', 'H11': 'L', 'H12': 'V', 'H13': 'N', 'H14': 'P', 'H15': 'G', 'H16': 'G', 'H17': 'S', 'H18': 'L', 'H19': 'R', 'H20': 'L', 'H21': 'S', 'H22': 'C', 'H23': 'V', 'H24': 'A', 'H25': 'S', 'H26': 'G', 'H27': 'F', 'H28': 'S', 'H29': 'F', 'H30': 'S', 'H31': 'H', 'H32': 'F', 'H33': 'D', 'H34': 'M', 'H35': 'N', 'H36': 'W', 'H37': 'V', 'H38': 'R', 'H39': 'Q', 'H40': 'A', 'H41': 'P', 'H42': 'G', 'H43': 'K', 'H44': 'G', 'H45': 'L', 'H46': 'Q', 'H47': 'W', 'H48': 'V', 'H49': 'A', 'H50': 'T', 'H51': 'I', 'H52': 'P', 'H52A': 'A', 'H53': 'D', 'H54': 'G', 'H55': 'S', 'H56': 'D', 'H57': 'T', 'H58': 'Y', 'H59': 'Y', 'H60': 'T', 'H61': 'G', 'H62': 'A', 'H63': 'V', 'H64': 'R', 'H65': 'G'

100%|█████████▉| 2995/3000 [03:50<00:00, 12.51it/s]

[{'H-FR1': 'EGQLAESGGDLVRPAGSLRLSCVAS', 'H-CDR1': 'GFTFSSYD', 'H-FR2': 'MTWVRQTPGKGLQWVAG', 'H-CDR2': 'IHSSGSYT', 'H-FR3': 'RYADAVKGRFTVSRDNAKNTVYLQMNSLRAEDTAVYYC', 'H-CDR3': 'ASPGGDYQDSFEY', 'H-FR4': 'WGQGTLVTVSS'}, {'H1': 'E', 'H2': 'G', 'H3': 'Q', 'H4': 'L', 'H5': 'A', 'H6': 'E', 'H7': 'S', 'H8': 'G', 'H9': 'G', 'H10': 'D', 'H11': 'L', 'H12': 'V', 'H13': 'R', 'H14': 'P', 'H15': 'A', 'H16': 'G', 'H17': 'S', 'H18': 'L', 'H19': 'R', 'H20': 'L', 'H21': 'S', 'H22': 'C', 'H23': 'V', 'H24': 'A', 'H25': 'S', 'H26': 'G', 'H27': 'F', 'H28': 'T', 'H29': 'F', 'H30': 'S', 'H31': 'S', 'H32': 'Y', 'H33': 'D', 'H34': 'M', 'H35': 'T', 'H36': 'W', 'H37': 'V', 'H38': 'R', 'H39': 'Q', 'H40': 'T', 'H41': 'P', 'H42': 'G', 'H43': 'K', 'H44': 'G', 'H45': 'L', 'H46': 'Q', 'H47': 'W', 'H48': 'V', 'H49': 'A', 'H50': 'G', 'H51': 'I', 'H52': 'H', 'H52A': 'S', 'H53': 'S', 'H54': 'G', 'H55': 'S', 'H56': 'Y', 'H57': 'T', 'H58': 'R', 'H59': 'Y', 'H60': 'A', 'H61': 'D', 'H62': 'A', 'H63': 'V', 'H64': 'K', 'H65': 'G'

100%|██████████| 3000/3000 [03:51<00:00, 12.98it/s]

[{'H-FR1': 'EGQLAESGGDLAKPGGSLRLSCVAS', 'H-CDR1': 'GLTLSRYS', 'H-FR2': 'MSWVRQAPGKGLQWVTD', 'H-CDR2': 'ISYDGSST', 'H-FR3': 'DYTDAVKGRFTISRDNARNTLYLEMNSLRAEDTAVYYC', 'H-CDR3': 'VPNPSSSCWSYFDY', 'H-FR4': 'WGQGTLVTVSS'}, {'H1': 'E', 'H2': 'G', 'H3': 'Q', 'H4': 'L', 'H5': 'A', 'H6': 'E', 'H7': 'S', 'H8': 'G', 'H9': 'G', 'H10': 'D', 'H11': 'L', 'H12': 'A', 'H13': 'K', 'H14': 'P', 'H15': 'G', 'H16': 'G', 'H17': 'S', 'H18': 'L', 'H19': 'R', 'H20': 'L', 'H21': 'S', 'H22': 'C', 'H23': 'V', 'H24': 'A', 'H25': 'S', 'H26': 'G', 'H27': 'L', 'H28': 'T', 'H29': 'L', 'H30': 'S', 'H31': 'R', 'H32': 'Y', 'H33': 'S', 'H34': 'M', 'H35': 'S', 'H36': 'W', 'H37': 'V', 'H38': 'R', 'H39': 'Q', 'H40': 'A', 'H41': 'P', 'H42': 'G', 'H43': 'K', 'H44': 'G', 'H45': 'L', 'H46': 'Q', 'H47': 'W', 'H48': 'V', 'H49': 'T', 'H50': 'D', 'H51': 'I', 'H52': 'S', 'H52A': 'Y', 'H53': 'D', 'H54': 'G', 'H55': 'S', 'H56': 'S', 'H57': 'T', 'H58': 'D', 'H59': 'Y', 'H60': 'T', 'H61': 'D', 'H62': 'A', 'H63': 'V', 'H64': 'K', 'H65': 'G

In [3]:
from Bio import SeqIO, SeqRecord
import os, contextlib
import pandas as pd
from tqdm import tqdm
import warnings
from Bio.pairwise2 import format_alignment
import json
from Bio import Align

warnings.filterwarnings("ignore")

small_ft_full_data_path = "../../datasets/outputs/small_ft_full_beam/test/processed"
small_ft_data_path = "../../datasets/outputs/small_ft_beam/test/processed"
small_ft_only_data_path = "../../datasets/outputs/small_ft_only_beam/test/processed"

scheme="martin"

i = 0
j = 0
loss = 0
heavy_chains = 0
light_chains = 0

records = []
seqs = []

cdr1s = json.load(open("cdr1s.json"))
cdr2s = json.load(open("cdr2s.json"))
cdr3s = json.load(open("cdr3s.json"))

aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.open_gap_score = -0
aligner.extend_gap_score = -0

with open(small_ft_full_data_path) as handle:
    for k, record in enumerate(tqdm(handle.readlines())):
        if k % 3 == 1:
            # print(record)
            seq=annotate(record,scheme)
            with open(os.devnull, 'w') as devnull:
                with contextlib.redirect_stdout(devnull):
                    # f()
                    result=seq.retrieve()
            # print(result)
            if result != None:
                j+=1
                # Compare CDRs
                if result[0].get("L-CDR1") is not None:
                    cdr1 = result[0]["L-CDR1"]
                    cdr2 = result[0]["L-CDR2"]
                    cdr3 = result[0]["L-CDR3"]
                    light_chains += 1
                else:
                    cdr1 = result[0]["H-CDR1"]
                    cdr2 = result[0]["H-CDR2"]
                    cdr3 = result[0]["H-CDR3"]
                    heavy_chains += 1
                alignment = aligner.align(cdr1, cdr1s[i])
                loss += len(cdr1s[i]) - alignment[0].score

                alignment = aligner.align(cdr2, cdr2s[i])
                loss += len(cdr2s[i]) - alignment[0].score

                alignment = aligner.align(cdr3, cdr3s[i])
                loss += len(cdr3s[i]) - alignment[0].score
                
            i+=1
print("Percentage separable for small_ft_full")
print(j/i)

print("average loss per separable sequence")
print(loss/j)
print("heavy chains")
print(heavy_chains)
print("light chains")
print(light_chains)

i = 0
j = 0
loss = 0
heavy_chains = 0
light_chains = 0

records = []
seqs = []

with open(small_ft_data_path) as handle:
    for k, record in enumerate(tqdm(handle.readlines())):
        if k % 3 == 1:
            seq=annotate(record,scheme)
            with open(os.devnull, 'w') as devnull:
                with contextlib.redirect_stdout(devnull):
                    # f()
                    result=seq.retrieve()
            if result != None:
                j+=1
                if result[0].get("L-CDR1") is not None:
                    cdr1 = result[0]["L-CDR1"]
                    cdr2 = result[0]["L-CDR2"]
                    cdr3 = result[0]["L-CDR3"]
                    light_chains += 1
                else:
                    cdr1 = result[0]["H-CDR1"]
                    cdr2 = result[0]["H-CDR2"]
                    cdr3 = result[0]["H-CDR3"]
                    heavy_chains += 1
                alignment = aligner.align(cdr1, cdr1s[i])
                loss += len(cdr1s[i]) - alignment[0].score

                alignment = aligner.align(cdr2, cdr2s[i])
                loss += len(cdr2s[i]) - alignment[0].score

                alignment = aligner.align(cdr3, cdr3s[i])
                loss += len(cdr3s[i]) - alignment[0].score
            i+=1

        
print("Percentage separable for small_ft")
print(j/i)
print("average loss per separable sequence")
print(loss/j)
print("heavy chains")
print(heavy_chains)
print("light chains")
print(light_chains)

i = 0
j = 0
loss = 0
heavy_chains = 0
light_chains = 0

records = []
seqs = []

with open(small_ft_only_data_path) as handle:
    for k, record in enumerate(tqdm(handle.readlines())):
        if k % 3 == 1:
            seq=annotate(record,scheme)
            with open(os.devnull, 'w') as devnull:
                with contextlib.redirect_stdout(devnull):
                    # f()
                    result=seq.retrieve()
            if result != None:
                j+=1
                if result[0].get("L-CDR1") is not None:
                    cdr1 = result[0]["L-CDR1"]
                    cdr2 = result[0]["L-CDR2"]
                    cdr3 = result[0]["L-CDR3"]
                    light_chains += 1
                else:
                    cdr1 = result[0]["H-CDR1"]
                    cdr2 = result[0]["H-CDR2"]
                    cdr3 = result[0]["H-CDR3"]
                    heavy_chains += 1
                alignment = aligner.align(cdr1, cdr1s[i])
                loss += len(cdr1s[i]) - alignment[0].score

                alignment = aligner.align(cdr2, cdr2s[i])
                loss += len(cdr2s[i]) - alignment[0].score

                alignment = aligner.align(cdr3, cdr3s[i])
                loss += len(cdr3s[i]) - alignment[0].score
            i+=1
print("Percentage separable for small_ft_only")
print(j/i)
print("average loss per separable sequence")
print(loss/j)

print("heavy chains")
print(heavy_chains)
print("light chains")
print(light_chains)

  1%|          | 28/3000 [00:02<03:57, 12.53it/s]


Percentage separable for small_ft_full
1.0
average loss per separable sequence
0.0
heavy chains
10
light chains
0


100%|██████████| 3000/3000 [03:51<00:00, 12.98it/s]


Percentage separable for small_ft
1.0
average loss per separable sequence
0.0
heavy chains
1000
light chains
0


100%|██████████| 3000/3000 [03:41<00:00, 13.54it/s]

Percentage separable for small_ft_only
0.841
average loss per separable sequence
0.0558858501783591
heavy chains
841
light chains
0
